In [1]:
from mandala._next.imports import *
from mandala._next.utils import *
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

storage = Storage(deps_path="__main__")

In [29]:
@op
def load_data(n_class):
    print('loading data')
    return load_digits(n_class=n_class, return_X_y=True)

@op
def train_model(X, y, n_estimators):
    print('training model')
    return RandomForestClassifier(max_depth=2, n_estimators=n_estimators).fit(X, y)

@op
def get_acc(model, X, y):
    print('computing accuracy')
    return round(model.score(X, y), 3)

In [ ]:
with storage:
    for n_class in (2, 5):
        X, y = load_data(n_class=n_class)
        for n_estimators in (10, 20):
            model = train_model(X, y, n_estimators)
            acc = get_acc(model, X, y)

Sounds great. I am now brainstorming the text for a very concise but effective Colab notebook introducing the main ideas of the library in a toy setting (so, the functions being memoized will be simple arithmetic ones to keep things short and not to distract from the main messages). 

For background: the library is based on two main constructs, an @op decorator, and the ComputationFrame class. The @op decorator is a memoizing decorator that also (optionally) tracks the code and dependencies of the function at the time it is called. Importantly, @op-decorated functions are designed to be composed with each other. The ComputationFrame is then used to navigate the shared storage of the @op functions as we have discussed. 

# A quick intro to `mandala`

# The `@op` decorator: memoization and versioning in a shared storage
`@op` tracks the inputs, outputs, code and dependencies of calls to Python
functions. The same call is never executed twice:

```python
from mandala._next.imports import *

storage = Storage(deps_path='__main__') # stores all `@op` calls

@op
def inc(x):
    print("Hello from inc!")
    return x + 1

with storage:
    a = inc(1)
    b = inc(1) # this will not be executed, but reused
```


# `ComputationFrame`s: generalized dataframes for program traces

`@op`s are designed to be composed with one another like ordinary Python
functions. This automatically keeps track of the relationships between all saved
objects. To explore these relationships, the `ComputationFrame` class is
provided:

```python
@op
def add(x, y):
    print("Hello from add!")
    return x + y

with storage:
    for i in range(5):
        j = inc(i)
        if i % 2 == 0:
            k = add(i, j)

cf = storage.cf(inc) # get the computation frame for all calls to `inc`
cf.draw(verbose=True, orientation='LR') # visualize the computation frame
cf.expand(inplace=True) # include all calls connected to the calls of `inc`
cf.draw(verbose=True, orientation='LR') # visualize the computation frame
```

Computation frames generalize the familiar dataframes:
- columns are replaced by the topology of a computational graph, consisting of variables and functions 
- rows are replaced by **computation traces** -- variable values and function calls -- that partially follow this graph

**Any computation frame can be converted into a dataframe**, where the columns are all nodes in the graph, and each row is a computation trace, possibly padded with `NaN`s where no value/call is present:

```python
cf.df()
```

Extracting tuples from the computation graph:
    output_0@output_0 = inc(x=x)
    output_0_0@output_0 = add(x=x, y=output_0)

+---+---------------------------------------+----------+---------------------------------------+------------+
| x |                  inc                  | output_0 |                  add                  | output_0_0 |
+---+---------------------------------------+----------+---------------------------------------+------------+
| 3 | Call(inc, cid='03d...', hid='2a9...') |    4     |                                       |            |
| 0 | Call(inc, cid='ad3...', hid='978...') |    1     | Call(add, cid='83a...', hid='955...') |    1.0     |
| 2 | Call(inc, cid='21f...', hid='82d...') |    3     | Call(add, cid='253...', hid='28e...') |    5.0     |
| 4 | Call(inc, cid='092...', hid='ea8...') |    5     | Call(add, cid='796...', hid='bb1...') |    9.0     |
| 1 | Call(inc, cid='35f...', hid='43a...') |    2     |                                       |            |
+---+---------------------------------------+----------+---------------------------------------+------------+